# Retrieval [Colab]

<font size=3>

Per l'ultimo task di retrieval sono stati utilizzati due notebook, questo è quello relativo allo script effettuato in Colab per ragioni computazionali

In [0]:
import keras
import os
import joblib
import numpy as np
import pickle
import sklearn
import tarfile
from tqdm import tqdm

from keras.preprocessing import image as kimage
from keras.applications import inception_v3
from keras.applications.inception_v3 import preprocess_input
from skimage import feature

Using TensorFlow backend.


In [0]:
def load_data(feature_extractor, base_path, maximages):
    paths = []
    features = []
    
    for fi,f in enumerate(tqdm(sorted(os.listdir(base_path)))):
        if f.endswith('.jpg') and fi < maximages:
            # Memorizza percorso file
            cur_path = base_path + f
            paths.append(cur_path)
            
            # Carica file ed estraine le features
            image = kimage.load_img(cur_path, target_size=(299, 299))
            cur_features = feature_extractor(image)
            features.append(cur_features)

    features = np.array(features)
    return features, paths



def nn_inception(img):
    x = kimage.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = inception_v3.preprocess_input(x)
    f = nn.predict(x)
    return f.flatten()

In [0]:
nn = inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling='max')

W0718 17:35:43.729918 140532589647744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 17:35:43.772158 140532589647744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 17:35:43.780449 140532589647744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 17:35:43.820990 140532589647744 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0718 17:35:43.822299 1405325896

96116736/96112376 [==============================] - 9s 0us/step


<font size=3>
    
I primi passaggi sono uguali ai precedenti punti, si parte dal caricamento delle immagini e dall'estrazione delle features mediante la rete neruale <b>InceptionV3</b>. 
<br></br>
Il dataset indicato sull'e-elearning nel progetto d'esame [non è più disponibile](https://www.microsoft.com/en-us/research/project/msra-cfw-data-set-of-celebrity-faces-on-the-web/), è stato quindi scaricato un dataset affine degli stessi autori reperibile a [questo indirizzo](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html). La differenza sostanziale è sul numero di persone presenti, oltre $10000$, mentre per il numero di file presenti la dimensione è pressoché la stessa $205$ mila contro le $202$ mila di quello consigliato.
<br></br>

I file sono stati caricati un dropbox, caricati ed estratti con i seguenti blocchi di codice.

In [0]:
!wget https://www.dropbox.com/s/rhe6vmiulqz7c8z/celebs.tar

tar = tarfile.open('celebs.tar')

--2019-07-18 17:36:13--  https://www.dropbox.com/s/rhe6vmiulqz7c8z/celebs.tar
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rhe6vmiulqz7c8z/celebs.tar [following]
--2019-07-18 17:36:14--  https://www.dropbox.com/s/raw/rhe6vmiulqz7c8z/celebs.tar
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7aa74bb29d0e763efa2607b4d5.dl.dropboxusercontent.com/cd/0/inline/Ak_rDRHUmPQNwVeK1K9l_6dtBCq9dgojKgmXbFOjCW22cDCsYO5dgrD5E_L5eQYDOolXWdG8ep6uMjGiPBBoyBIEHId0Gfe9pI3DnpiUQ334IA/file# [following]
--2019-07-18 17:36:14--  https://uc7aa74bb29d0e763efa2607b4d5.dl.dropboxusercontent.com/cd/0/inline/Ak_rDRHUmPQNwVeK1K9l_6dtBCq9dgojKgmXbFOjCW22cDCsYO5dgrD5E_L5eQYDOolXWdG8ep6uMjGiPBBoyBIEHId0Gfe9pI3DnpiUQ334IA/file
Resolving uc7aa74bb29d

In [0]:
tar.extractall()
tar.close()

base_path = './celebs/'

<font size=3>
    
Viene effettuata l'estrazione delle features. Con l'acceleratore di GPU il processo è stato terminato in poco più di due ore, mentre nei notebook sulle nostre macchine il processo effettuava solamente $2$ iterazioni al secondo con un tempo stimato di oltre un giorno, da qui la necessità di utilizzare Colab.

In [0]:
X_train, paths = load_data(feature_extractor=nn_inception, base_path=base_path, maximages=1000000)

100%|██████████| 202599/202599 [2:16:57<00:00, 24.66it/s]


<font size=3>
    
La parte finale dello script è per il salvataggio dei file riguardanti le feature estratte e i percorsi delle immagini relative (questa lista è necessaria per stampare a video le immagini nell'altro notebook) su Colab e il successivo download in locale.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
np.save('Xtrain.npy', X_train_norm)

In [0]:
from google.colab import files
files.download('Xtrain.npy') 

In [0]:
with open('paths_list', 'wb') as fp:
    pickle.dump(paths, fp)